In [4]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Dense, Input, Dropout, Flatten
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import LabelEncoder
from sklearn.decomposition import PCA
from sklearn.metrics import classification_report
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from keras.datasets import cifar100
from keras.utils import to_categorical
from keras.layers import Conv2D, MaxPooling2D

In [5]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("alxmamaev/flowers-recognition")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/flowers-recognition


In [6]:
datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.3,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2
)

In [7]:
train = datagen.flow_from_directory(
    '/kaggle/input/flowers-recognition/flowers',
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical',
    subset='training',
    shuffle=True
)

val = datagen.flow_from_directory(
    '/kaggle/input/flowers-recognition/flowers',
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical',
    subset='validation',
    shuffle=False
)

Found 3457 images belonging to 5 classes.
Found 860 images belonging to 5 classes.


In [11]:
CNN_model = Sequential()
CNN_model.add(Input(shape=(128, 128, 3)))
CNN_model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
CNN_model.add(MaxPooling2D((2, 2)))
CNN_model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
CNN_model.add(MaxPooling2D((2, 2)))
CNN_model.add(Conv2D(16, (3, 3), activation='relu', padding='same'))
CNN_model.add(MaxPooling2D((2, 2)))
CNN_model.add(Flatten())

CNN_model.add(Dense(64,activation='relu'))
CNN_model.add(Dense(5, activation='softmax'))
CNN_model.compile(optimizer='adam', loss='categorical_crossentropy',metrics=['accuracy'])

In [12]:
CNN_model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_5 (Conv2D)               │ (None, 128, 128, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 64, 64, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_6 (Conv2D)               │ (None, 64, 64, 32)     │        18,464 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_6 (MaxPooling2D)  │ (None, 32, 32, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 32, 32, 16)     │         4,624 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_7 (MaxPooling2D)  │ (None, 16, 16, 16)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ (None, 4096)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 64)             │       262,208 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 5)              │           325 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 287,413 (1.10 MB)

 Trainable params: 287,413 (1.10 MB)

 Non-trainable params: 0 (0.00 B)

In [13]:
history = CNN_model.fit(train, validation_data=val,epochs=10)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
109/109 ━━━━━━━━━━━━━━━━━━━━ 35s 283ms/step - accuracy: 0.2630 - loss: 1.5576 - val_accuracy: 0.5372 - val_loss: 1.1874
Epoch 2/10
109/109 ━━━━━━━━━━━━━━━━━━━━ 26s 235ms/step - accuracy: 0.5500 - loss: 1.1350 - val_accuracy: 0.5709 - val_loss: 1.0510
Epoch 3/10
109/109 ━━━━━━━━━━━━━━━━━━━━ 25s 234ms/step - accuracy: 0.5979 - loss: 1.0461 - val_accuracy: 0.5919 - val_loss: 1.0499
Epoch 4/10
109/109 ━━━━━━━━━━━━━━━━━━━━ 25s 233ms/step - accuracy: 0.5782 - loss: 1.0414 - val_accuracy: 0.6209 - val_loss: 0.9750
Epoch 5/10
109/109 ━━━━━━━━━━━━━━━━━━━━ 25s 232ms/step - accuracy: 0.6463 - loss: 0.9219 - val_accuracy: 0.6291 - val_loss: 0.9229
Epoch 6/10
109/109 ━━━━━━━━━━━━━━━━━━━━ 25s 232ms/step - accuracy: 0.6384 - loss: 0.9315 - val_accuracy: 0.6105 - val_loss: 0.9644
Epoch 7/10
109/109 ━━━━━━━━━━━━━━━━━━━━ 26s 236ms/step - accuracy: 0.6352 - loss: 0.9339 - val_accuracy: 0.6663 - val_loss: 0.8853
Epoch 8/10
109/109 ━━━━━━━━━━━━━━━━━━━━ 31s 282ms/step - accuracy: 0.6843 - loss: 0